#### passing parameters to the notebook (adding a widget)

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

#### invoke a notebook from another notebook using the %run command

In [0]:
%run "../Includes/configuration"

In [0]:
%run  "../Includes/common_functions"

In [0]:
pit_stops_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                     StructField("driverId", IntegerType(), True),
                                     StructField("stop", IntegerType(), True),
                                     StructField("lap", IntegerType(), True),
                                     StructField("time", StringType(), True),
                                     StructField("duration", StringType(), True),
                                     StructField("milliseconds", StringType(), True)])

In [0]:
pit_stops_df = spark.read \
.option("multiLine", True)\
.schema(pit_stops_schema) \
.json(f"{raw_folder_path}/{v_file_date}/pit_stops.json")

In [0]:
pit_stops_final_df = pit_stops_df.withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("driverId", "driver_id") \
.withColumn("ingestion_date", current_timestamp())\
.withColumn("pit_stops_id", col("stop").cast("integer"))\
.withColumn("data_source", lit(v_data_source))\
.withColumn("file_date", lit(v_file_date))


In [0]:
#for race_id_list in pit_stops_final_df.select("race_id").distinct().collect():
#   if (spark._jsparkSession.catalog().tableExists("f1_processed.pit_stops")):
#    spark.sql(f"ALTER TABLE f1_processed.pit_stops DROP IF EXISTS PARTITION (race_id = {race_id_list.race_id})")
#pit_stops_final_df.write.mode("append").partitionBy("race_id").format("parquet").saveAsTable("f1_processed.pit_stops")


##### incremental Load for Delta files

In [0]:
merge_condition = "tgt.stop = src.stop AND tgt.driver_id = src.driver_id AND tgt.race_id = src.race_id"
merge_delta_data(pit_stops_final_df, "f1_processed", "pit_stops", processed_folder_path, merge_condition, "race_id")

In [0]:
dbutils.notebook.exit("Success")